# Likelihood for Restaurant Business Success (in New York vs. in Toronto)

## Business Problem
I am an aspirational chef who is considering to migrate to a vibrant city.
The choices are narrowed down to two amazing cities, New York or Toronto.

I cannot afford to fail as I am putting my life saving into the business.
I am going to explore the likelihood of success, using my newly acquired skillset of Data Science!

## Key Metric
We are going to observe and compare two data features, 
1. the number of restaurants between the two cities
2. the variations among these restaurants

## Data Source
We are going to rely on FourSquare dataset for both cities
1. We are going to count the number and types of restaurant in both cities